In [1]:
import os

from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from snowflake.sqlalchemy import URL
from langchain import SQLDatabase

# Load environments variables
from dotenv import load_dotenv
load_dotenv("../.streamlit/secrets.toml") 


# create snowflake connection
uri_snow = URL(
    account=os.getenv("account"),
    user=os.getenv("user"),
    password=os.getenv("password"),
    database=os.getenv("database"),
    schema=os.getenv("schema"),
    warehouse=os.getenv("warehouse"),
    role=os.getenv("role"),
)

tables = ["contact", "employee", "priority_contact", "readership", "interaction"]
db = SQLDatabase.from_uri(uri_snow, include_tables=tables)
# db = SQLDatabase.from_uri(uri_snow)
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

In [2]:
agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [4]:
agent_executor("top 5 contacts name who read most")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: contact, employee, interaction, priority_contact, readership
Thought: I should query the schema of the contact and readership tables.
Action: sql_db_schema
Action Input: contact, readership
Observation: 
CREATE TABLE contact (
	contact_key DECIMAL(38, 0) NOT NULL, 
	contact_id VARCHAR(30), 
	contact_name VARCHAR(100), 
	client_id VARCHAR(30), 
	client_name VARCHAR(100), 
	tier VARCHAR(10), 
	region VARCHAR(50), 
	active_flag VARCHAR(1), 
	CONSTRAINT "SYS_CONSTRAINT_4bb18bf2-e5e8-4cf0-b4eb-ad7e20161aef" PRIMARY KEY (contact_key)
)

/*
3 rows from contact table:
contact_key	contact_id	contact_name	client_id	client_name	tier	region	active_flag
1	CT101	John Doe	CL001	ABC Corp	Gold	North America	Y
2	CT102	Jane Smith	CL002	XYZ Corp	Silver	Europe	Y
3	CT103	Mike Johnson	CL003	123 Industries	Bronze	Asia	Y
*/


CREATE TABLE readership (
	readership_key DECIMAL(38, 0) NOT NULL, 
	document_key DECIMAL(38

{'input': 'top 5 contacts name who read most',
 'output': 'The top 5 contacts who read the most are Sophia Smith (306), James Lee (161), Olivia Kim (161), William Smith (160), and Noah Johnson (159).'}